In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
# library(readr)
library(ggplot2)
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
# options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")
future::plan()

### 质量控制并确定变异基因

In [ ]:
# qread速度很快
library(qs)
system.time({
    seurat.data = qread(file = "./Outdata/Step3.Cluster_annotion.qs")
           })

In [ ]:
table(seurat.data@meta.data$celltype)

#### 选择需要二次聚类的细胞类型

In [ ]:
## 选择需要二次聚类的细胞类型
seurat.data = subset(seurat.data, celltype %in% c("Macrophage"))

In [ ]:
#使用PercentageFeatureSet函数计算线粒体基因的百分比
seurat.data[["percent.mt"]] <- PercentageFeatureSet(object = seurat.data, pattern = "^mt-")
pdf(file="04.featureViolin.pdf",width=10,height=6)           #保存基因特征小提琴图
VlnPlot(object = seurat.data, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3,group.by = "group")
dev.off()
seurat.data <- subset(x = seurat.data, subset = nFeature_RNA > 50 & percent.mt < 5)    #对数据进行过滤


In [ ]:
#测序深度的相关性绘图
pdf(file="04.featureCor.pdf",width=10,height=6)              #保存基因特征相关性图
plot1 <- FeatureScatter(object = seurat.data, feature1 = "nCount_RNA", feature2 = "percent.mt",pt.size=1.5,group.by = "group")
plot2 <- FeatureScatter(object = seurat.data, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",,pt.size=1.5,group.by = "group")
CombinePlots(plots = list(plot1, plot2))
dev.off()


In [ ]:
#对数据进行标准化
seurat.data <- NormalizeData(object = seurat.data, normalization.method = "LogNormalize", scale.factor = 10000)
#提取那些在细胞间变异系数较大的基因
seurat.data <- FindVariableFeatures(object = seurat.data, selection.method = "vst", nfeatures = 2000)



In [ ]:
seurat.data

In [ ]:
#输出特征方差图
top10 <- head(x = VariableFeatures(object = seurat.data), 10)
pdf(file="04.featureVar.pdf",width=10,height=6)              #保存基因特征方差图
plot1 <- VariableFeaturePlot(object = seurat.data)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
CombinePlots(plots = list(plot1, plot2))
dev.off()

In [ ]:
pcSelect=20

##PCA分析
seurat.data=ScaleData(seurat.data)                     #PCA降维之前的标准预处理步骤
seurat.data=RunPCA(object= seurat.data,npcs = pcSelect,pc.genes=VariableFeatures(object = seurat.data))     #PCA分析


In [ ]:
seurat.data = seurat.data %>% 
    RunUMAP(reduction = "pca", dims = 1:pcSelect, verbose = F)

### 去批次

In [ ]:
### 3.2 检查批次
options(repr.plot.width = 10, repr.plot.height = 4.5)
p1.compare=wrap_plots(ncol = 2,
                      DimPlot(seurat.data, reduction = "pca", group.by = "sampleID")+NoAxes()+ggtitle("Before_PCA"),
                      DimPlot(seurat.data, reduction = "umap", group.by = "sampleID")+NoAxes()+ggtitle("Before_UMAP"),
                      guides = "collect"
)
p1.compare

In [ ]:
### 4.1 RunHarmony 小样本运行比较快
library(harmony)
seurat.data <- seurat.data %>% RunHarmony("sampleID", plot_convergence = T)

In [ ]:
### 去批次之后，还需要再RunUMAP一次更新harmony
seurat.data <- seurat.data %>% 
  RunUMAP(reduction = "harmony", dims = 1:pcSelect, verbose = F)

In [ ]:
p2.compare=wrap_plots(ncol = 2,
                      DimPlot(seurat.data, reduction = "harmony", group.by = "sampleID")+NoAxes()+ggtitle("After_PCA (harmony)"),
                      DimPlot(seurat.data, reduction = "umap", group.by = "sampleID")+NoAxes()+ggtitle("After_UMAP"),
                      guides = "collect"
)
# p2.compare

options(repr.plot.width = 10, repr.plot.height = 9)
wrap_plots(p1.compare, p2.compare, ncol = 1)


In [ ]:
pdf(file="after_hamrmony.pdf",width=10,height=9)
wrap_plots(p1.compare, p2.compare, ncol = 1)
dev.off()

### 找合适的resolution

In [ ]:
# 对比多种resolution的聚类结果
seurat.data <- FindNeighbors(seurat.data,reduction = "harmony", dims = 1:pcSelect)
for (res in c(0.05,0.1,0.2,0.3,0.5,0.8,1,1.2,1.4,1.5,2)){
  print(res)
  seurat.data <- FindClusters(seurat.data,resolution = res, algorithm = 1)
}

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
#umap可视化
cluster_umap <- wrap_plots(ncol = 5,
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.05", label = T) & NoAxes(),  
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.1", label = T) & NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.2", label = T) & NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.3", label = T)& NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.5", label = T) & NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.0.8", label = T) & NoAxes(), 
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.1", label = T) & NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.1.2", label = T) & NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.1.4", label = T)& NoAxes(),
                           DimPlot(seurat.data, reduction = "umap", group.by = "RNA_snn_res.1.5", label = T)& NoAxes()
)
cluster_umap

In [ ]:
Idents(seurat.data) <- seurat.data@meta.data$RNA_snn_res.0.2

In [ ]:
pdf(file="07.Umap_0.2.pdf",width=6.5,height=6)
# 默认用seurat_clusters列（最后一次聚类得到的列）
DimPlot(seurat.data, reduction = "umap", label = TRUE) & NoAxes()
dev.off()

In [ ]:
#qs速度快
#install.packages('qs')
library(qs)
system.time({
    qsave(seurat.data,file = "./Outdata/Cluster_no_annotion_End.qs") 
})

## 按照差异基因0.2的清晰度聚类

In [ ]:
# qread速度很快
library(qs)
system.time({
    seurat.data = qread(file = "./Outdata/Sub_annotion.qs")
           })

In [ ]:
seurat.data <- subset(seurat.data, subset = !grepl("\\(", celltype))


In [ ]:
head(seurat.data@meta.data)

In [ ]:
Idents(seurat.data) <- seurat.data@meta.data$RNA_snn_res.0.2

In [ ]:
##寻找差异表达的特征
log2FCfilter=1 # 表示2的1次方，即相差两倍
adjPvalFilter=0.05

In [ ]:

seurat.data.markers <- FindAllMarkers(object = seurat.data,
                               only.pos = FALSE,
                               min.pct = 0.25,
                               logfc.threshold = log2FCfilter)


In [ ]:
# 选择前3个基因z作为每个cluster的标志基因
top3 <- seurat.data.markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC)


In [ ]:
#绘制差异基因在各个细胞类型的热图（图片有问题）
pdf(file="06.Heatmap.pdf",width=25,height=15)
DoHeatmap(seurat.data, features = top3$gene, 
# group.by = "RNA_snn_res.0.2"
)+ 
#   RotatedAxis() +  # 旋转 X 轴文字，推荐 Seurat 自带函数
  theme(
    # axis.text.x = element_text(size = 16, angle = 45),  # 横轴文字
    axis.text.y = element_text(size = 18),                          # 纵轴文字
    # axis.title.x = element_blank(),  # 去掉X轴标题
    # axis.title.y = element_blank(),  # 去掉Y轴标题
    legend.title = element_text(size = 18),
    legend.text  = element_text(size = 18),
    legend.key.size = unit(1, "cm")  # 图例符号大小
  )
dev.off()

In [ ]:
save(seurat.data.markers,
     file = "Step6.celltype.markers.0.2_macro.Rdata")

In [ ]:
sig.markers=seurat.data.markers[(abs(as.numeric(as.vector(seurat.data.markers$avg_log2FC)))>log2FCfilter & as.numeric(as.vector(seurat.data.markers$p_val_adj))<adjPvalFilter),]
write.table(sig.markers,file="06.markers_macro.xls",sep="\t",row.names=F,quote=F)


In [ ]:
head(sig.markers)

### 主要细胞类型注释

In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
# library(readr)
library(ggplot2)
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
# options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")
future::plan()

In [ ]:
# 读取未注释的数据
seurat.data = qread(file = "./Outdata/Cluster_no_annotion_End.qs")

In [ ]:
check_genes = c(
                "Adgre1","Fcgr1","Cd68", # 巨噬细胞
                "Cpa3","Hpgds","Ms4a2", # 肥大细胞
                "Clec10a","Clec4c", #DC cells
                "Retnlg","Fcer1g","Cd14",   # 髓系细胞 (Myeloid_cells)  需要进一步细分巨噬细胞、中性粒细胞
                "Col1a1","Col1a2","Dcn",# 成纤维细胞 Fibroblast
                "Epcam","Cdh1","Krt18",   # 上皮细胞 (Epithelial cells)
                "S100a9", "S100a8",'Csf3r',"Fcgr3b", #"Mki67", #中性粒细胞
                "Cd79a", "Ms4a1","Cd19","Igkc", #B细胞
                "Acta2", "Myh11", #平滑肌细胞 Smooth muscle cells
                "Cd3d","Cd3g", #T细胞
                "Nkg7","Gzma","Ccl5",   # NK细胞 (NK_cells)
                "Ppbp","Gp1bb", # Platelets
                "Cldn5","Pecam1","Ramp2"   # 内皮细胞 (Endothelial cells)
)


DotPlot(object = seurat.data, features = check_genes, 
        assay = "RNA",scale = T) + coord_flip()

In [ ]:
###分配细胞名称
celltype=data.frame(ClusterID=0:12,celltype='NA')

## Neutrophils_c1
celltype[celltype$ClusterID %in% c(0),2]='Macrophages_c0' #CRL

## Macrophages
celltype[celltype$ClusterID %in% c(1),2]='Macrophages_c1' #CRL

## Macrophages_cells
celltype[celltype$ClusterID %in% c(2),2]='Macrophages_c2' #CRL

## B_cells
celltype[celltype$ClusterID %in% c(3),2]='Macrophages_c3(Epi)' # 还有毒性T细胞 #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(4),2]='Macrophages_c4' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(5),2]='Macrophages_c5' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(6),2]='Macrophages_c6' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(7),2]='Macrophages_c7(Neu)' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(8),2]='Macrophages_c8(NK/T)' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(9),2]='Macrophages_c9' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(10),2]='Macrophages_c10' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(11),2]='Macrophages_c11' #CRL

## Smooth muscle cells
celltype[celltype$ClusterID %in% c(12),2]='Macrophages_c12(Neu)' #CRL


colnames(celltype) = c("ClusterID","celltype_main")
seurat.data@meta.data$celltype = "NA"
for(i in 1:nrow(celltype)){
  seurat.data@meta.data[which(seurat.data@active.ident == celltype$ClusterID[i]),'celltype'] <- celltype$celltype[i]}
table(seurat.data@meta.data$celltype)

In [ ]:
pdf(file="Sub_End_Umap_annotion_0.2.pdf",width=12,height=10)
DimPlot(seurat.data, reduction = "umap", group.by = "celltype", label = T)& NoAxes()
dev.off()

In [ ]:
head(seurat.data@meta.data)
Idents(seurat.data) <- seurat.data@meta.data$celltype
table(seurat.data@meta.data$celltype)

In [ ]:
## 2.5 保存数据
qsave(seurat.data, file = "./Outdata/Sub_annotion.qs")

### 按照指定细胞类型顺序绘制Marker基因图

In [ ]:
# 按照指定顺序定义因子水平
celltype_order <- c(
  "Macrophages_c0",
  "Macrophages_c1",
  "Macrophages_c2",
  "Macrophages_c3(Epi)",
  "Macrophages_c4",
  "Macrophages_c5",
  "Macrophages_c6",
  "Macrophages_c7(Neu)",
  "Macrophages_c8(NK/T)",
  "Macrophages_c9",
  "Macrophages_c10",
  "Macrophages_c11",
  "Macrophages_c12(Neu)",
  "Macrophages_c13"
)
# 确保 celltype 列为因子
seurat.data$celltype <- factor(seurat.data$celltype, levels = celltype_order)

# DotPlot
p <- DotPlot(
    seurat.data, 
    features = check_genes, 
    assay = "RNA", 
    scale = TRUE,
    group.by = "celltype"
) + 
  RotatedAxis() +  # 旋转 X 轴文字，推荐 Seurat 自带函数
  theme(
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),  # 横轴文字
    axis.text.y = element_text(size = 16),                          # 纵轴文字
    axis.title.x = element_blank(),  # 去掉X轴标题
    axis.title.y = element_blank(),  # 去掉Y轴标题
    legend.title = element_text(size = 16),
    legend.text  = element_text(size = 16),
    legend.key.size = unit(0.6, "cm")  # 图例符号大小
  )

# 保存
pdf(file="Macro_reordered_easy.pdf", width=14, height=8)
print(p)
dev.off()


In [ ]:
library(Seurat)
library(ggplot2)

pdf(file="Macro_0.5_easy.pdf", width=12, height=8)

p <- DimPlot(
    seurat.data, 
    reduction = "umap", 
    group.by = "celltype", 
    label = TRUE, 
    label.size = 6
) & 
  NoAxes() &  
  theme(
    legend.title   = element_text(size = 16),   # 图例标题
    legend.text    = element_text(size = 16),   # 图例文字
  )

print(p)
dev.off()

## Marker基因可视化

In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
# library(readr)
library(ggplot2)
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
# options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")
future::plan()

In [ ]:
seurat.data=qread(file = "./Outdata/Sub_annotion.qs")

In [ ]:
seurat.data <- subset(seurat.data, subset = !grepl("\\(", celltype))


In [ ]:
seurat.data  <- subset(seurat.data, subset = celltype %in% c("Macrophages_c2", "Macrophages_c9", "Macrophages_c10"))

In [ ]:
levels(seurat.data)

In [ ]:
pdf(file="06.markerViolin_M1M2.pdf",width=12,height=8)
cluster10Marker=c("Cd86","Nos2","Cxcl9","Cxcl10","Cd163","Arg1","Mrc1","Chil3")
DotPlot(object = seurat.data, features = cluster10Marker) + theme(
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),  # 横轴文字
    axis.text.y = element_text(size = 16),                          # 纵轴文字
    axis.title.x = element_blank(),  # 去掉X轴标题
    axis.title.y = element_blank(),  # 去掉Y轴标题
    legend.title = element_text(size = 16),
    legend.text  = element_text(size = 16),
    legend.key.size = unit(0.8, "cm")  # 图例符号大小
  )
dev.off ()

In [ ]:


# # 绘制marker的小提琴图
pdf(file="06.markerViolin.pdf",width=12,height=8)
VlnPlot(object = seurat.data, features = c("Clec4e"),pt.size = 0)+
# theme(text = element_text(size = 16))
theme(
    axis.text.x = element_text(size = 16),   # x 轴刻度字体
    axis.text.y = element_text(size = 16),   # y 轴刻度字体
    axis.title.x = element_text(size = 16),  # x 轴标题
    axis.title.y = element_text(size = 16),  # y 轴标题
    plot.title  = element_text(size = 16, hjust = 0.5), # 主标题
    legend.text  = element_text(size = 16),
    legend.key.size = unit(1.3, "cm")
  )
dev.off()


In [ ]:


#绘制marker在各个cluster的散点图
pdf(file="06.markerScatter_cytokines.pdf",width=15,height=8)
FeaturePlot(object = seurat.data, features = c("Tlr2","Cxcl2","Lcn2","Il1b"),cols = c("gray", "red"), label = T, ncol = 2,label.size = 6)& NoAxes()+
theme(
    # axis.text.x = element_text(size = 16),   # x 轴刻度字体
    # axis.text.y = element_text(size = 16),   # y 轴刻度字体
    # axis.title.x = element_text(size = 16),  # x 轴标题
    # axis.title.y = element_text(size = 16),  # y 轴标题
    plot.title  = element_text(size = 16, hjust = 0.5), # 主标题
    legend.text  = element_text(size = 16),
    legend.key.size = unit(1, "cm")
  )
dev.off()


In [ ]:

# #绘制marker在各个cluster的气泡图
pdf(file="06.markerBubble.pdf",width=12,height=6)
cluster10Marker=c("Nos2","Cxcl9","Cxcl10","Arg1","Mrc1","Chil3")
DotPlot(object = seurat.data, features = cluster10Marker) + theme(
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),  # 横轴文字
    axis.text.y = element_text(size = 16),                          # 纵轴文字
    axis.title.x = element_blank(),  # 去掉X轴标题
    axis.title.y = element_blank(),  # 去掉Y轴标题
    legend.title = element_text(size = 16),
    legend.text  = element_text(size = 16),
    legend.key.size = unit(0.8, "cm")  # 图例符号大小
  )
dev.off()


### 富集分析

基因SYMBOL转GENE ID(富集的前提)

In [ ]:
# 重新加载数据
load("Step6.celltype.markers.0.2_macro.Rdata")

In [ ]:
##寻找差异表达的特征
log2FCfilter=1 # 表示2的1次方，即相差两倍
adjPvalFilter=0.05

head(seurat.data.markers)

In [ ]:
sig.markers=seurat.data.markers[(abs(as.numeric(as.vector(seurat.data.markers$avg_log2FC)))>log2FCfilter & as.numeric(as.vector(seurat.data.markers$p_val_adj))<adjPvalFilter),]


In [ ]:
library("org.Mm.eg.db")          #引用包小鼠
# rt=read.table("symbol.txt",sep="\t",check.names=F,header=T)    #读取文件
genes=as.vector(sig.markers[,7])
entrezIDs <- mget(genes, org.Mm.egSYMBOL2EG, ifnotfound=NA)    #找出基因对应的id
entrezIDs <- as.character(entrezIDs)
sig.markers_entrezIDs=cbind(sig.markers,entrezID=entrezIDs)
write.table(sig.markers_entrezIDs,file="./shengxinzixue/id.txt",sep="\t",quote=F,row.names=F)    #输出结果

In [ ]:
table(sig.markers_entrezIDs$cluster)

### GO富集（Macrophages_c10类）

In [ ]:
### 选择cluster为0的亚群进行富集分析
library("clusterProfiler")

In [ ]:
sig.markers_entrezIDs <- subset(sig.markers_entrezIDs, cluster == "Macrophages_c10") 
 
sig.markers_entrezIDs=sig.markers_entrezIDs[is.na(sig.markers_entrezIDs[,"entrezID"])==F,]                                 #去除基因id为NA的基因
gene=sig.markers_entrezIDs$entrezID

#GO富集分析
kk <- enrichGO(gene = gene,
               OrgDb = org.Mm.eg.db, 
               pvalueCutoff =0.05, 
               qvalueCutoff = 0.05,
               ont="all",
               readable =T)
write.table(kk,file="./shengxinzixue/GO_c10.txt",sep="\t",quote=F,row.names = F)                 #保存富集结果

In [ ]:
library(enrichplot)
library(ggplot2)
library(grid)  # 用于 unit()

# 输出 PDF
pdf(file="GO_barplot_c10.pdf", width=8, height=12)

# 绘图
barplot(kk,
        drop = TRUE,
        showCategory = 5,  #控制柱子的数量
        split = "ONTOLOGY") +
  facet_grid(ONTOLOGY ~ ., scales = "free") +   # 注意是 scales = "free"
  theme_minimal() +  # 可以换成 theme_bw() 或 theme_classic() 根据喜好
  theme(
    axis.text.x = element_text(color = "black", size = 16, angle = 45, hjust = 1), # x轴文字旋转更美观
    axis.text.y = element_text(color = "black", size = 16),
    axis.title = element_text(color = "black", size = 20),
    strip.text.y = element_text(color = "black", size = 20),  # 分面标签
    legend.text = element_text(size = 16),
    legend.title = element_text(size = 16),
    legend.key.size = unit(1, "cm")
  )

dev.off()


In [ ]:
pdf(file="GO_bubble_c10.pdf", width = 8, height = 12)

dotplot(kk, showCategory = 5, split = "ONTOLOGY") +
  facet_grid(ONTOLOGY ~ ., scale = "free") +
  theme(
    axis.text.x = element_text(color = "black", size = 16),   # x轴文字
    axis.text.y = element_text(color = "black", size = 16),   # y轴文字
    strip.text.y = element_text(color = "black", size = 20),  # 分面标签
    axis.title  = element_text(color = "black", size = 20),   # 坐标轴标题
    legend.text = element_text(size = 16),                    # 图例文字
    legend.title= element_text(size = 16),                    # 图例标题
    legend.key.size = unit(1, "cm")                           # 图例方块大小
  )

dev.off()


## KEGG (Macrophages_c2类)

In [ ]:
#kegg富集分析
kk <- enrichKEGG(gene = gene,
                 organism = "mmu",
                 pvalueCutoff = 0.05,
                 qvalueCutoff = 0.05)

write.table(kk,file="./shengxinzixue/KEGGId_2.txt",sep="\t",quote=F,row.names = F)                          #保存富集结果

In [ ]:
#柱状图
pdf(file="KEGG_barplot_c2.pdf",width = 10,height = 20)
# options(repr.plot.width = 10, repr.plot.height = 15)
barplot(kk, drop = TRUE, showCategory = 30) +
  theme(
    axis.text.x = element_text(color = "black", size = 16),  # x轴文字
    axis.text.y = element_text(color = "black", size = 16),    # y轴文字
    strip.text.y = element_text(color = "black", size = 20),  # 分面标签
    axis.title = element_text(color = "black", size = 20) # 坐标轴标题
  )+
    theme(
    legend.text = element_text(size = 16),   # 图例文字大小
    legend.title = element_text(size = 16),  # 图例标题大小
    legend.key.size = unit(1, "cm")          # 图例方块大小
  )

dev.off()



In [ ]:
pdf(file="KEGG_bubble_c2.pdf", width = 10, height = 20)

dotplot(kk, showCategory = 30) +
  theme(
    axis.text.x = element_text(color = "black", size = 16),   # x轴文字
    axis.text.y = element_text(color = "black", size = 16),   # y轴文字
    strip.text.y = element_text(color = "black", size = 20),  # 分面标签
    axis.title  = element_text(color = "black", size = 20),   # 坐标轴标题
    legend.text = element_text(size = 16),                    # 图例文字
    legend.title= element_text(size = 16),                    # 图例标题
    legend.key.size = unit(1, "cm")                           # 图例方块大小
  )

dev.off()
